## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [11]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

"wget" non Š riconosciuto come comando interno o esterno,
 un programma eseguibile o un file batch.


1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [12]:
import numpy as np
import pandas as pd

N = 1310720

df = pd.read_csv('data_000637.txt', nrows = 100000)
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25


In [13]:
X = df['BX_COUNTER'].max()
print("maximum value found: ", X)

maximum value found:  3563


In [14]:
d3 = df['ORBIT_CNT'].iloc[99999].max() - df['ORBIT_CNT'].iloc[0].min()
d2 = df['BX_COUNTER'].iloc[99999].max() - df['BX_COUNTER'].iloc[0].min()
d1 = df['TDC_MEAS'].iloc[99999].max() - df['TDC_MEAS'].iloc[0].min()

time = d3 *X * 25 + d2*25 + d1*25/30
print("time taken for 100k samples: ", time, " nanoseconds")
print("time estimated for entire dataset: ", time*13.1, " nanoseconds")

time taken for 100k samples:  88545313.33333333  nanoseconds
time estimated for entire dataset:  1159943604.6666665  nanoseconds


In [15]:
df['NANOSECONDS'] = df['ORBIT_CNT'] *X * 25 + df['BX_COUNTER']*25 + df['TDC_MEAS']*25/30
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,NANOSECONDS
0,1,0,123,3869200167,2374,26,3.446490e+14
1,1,0,124,3869200167,2374,27,3.446490e+14
2,1,0,63,3869200167,2553,28,3.446490e+14
3,1,0,64,3869200167,2558,19,3.446490e+14
4,1,0,64,3869200167,2760,25,3.446490e+14


In [16]:
df.loc[:,'HEAD'] = np.random.randint(2, size=100000)
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,NANOSECONDS
0,0,0,123,3869200167,2374,26,3.446490e+14
1,1,0,124,3869200167,2374,27,3.446490e+14
2,1,0,63,3869200167,2553,28,3.446490e+14
3,1,0,64,3869200167,2558,19,3.446490e+14
4,1,0,64,3869200167,2760,25,3.446490e+14


In [17]:
df2=df.copy()
df2 = df2[df2['HEAD']==1]
df2.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,NANOSECONDS
1,1,0,124,3869200167,2374,27,3.446490e+14
2,1,0,63,3869200167,2553,28,3.446490e+14
3,1,0,64,3869200167,2558,19,3.446490e+14
4,1,0,64,3869200167,2760,25,3.446490e+14
9,1,0,60,3869200167,2788,7,3.446490e+14


In [18]:
df2[df2['FPGA']==0].hist(column='TDC_CHANNEL')
df2[df2['FPGA']==1].hist(column='TDC_CHANNEL')

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x00000238DDD79B38>]],
      dtype=object)

In [19]:
df2.groupby('TDC_CHANNEL').size().nlargest(3)

TDC_CHANNEL
139    4171
63     2460
64     2435
dtype: int64

In [20]:
u1 = df['ORBIT_CNT'].nunique()
u2 = df[df['TDC_CHANNEL']==139]['ORBIT_CNT'].nunique()
print("unique orbit: ", u1)
print("unique orbit with tdc channel 139: ", u2)

unique orbit:  995
unique orbit with tdc channel 139:  994
